In [1]:
import numpy as np
import sympy as sp
from sympy import latex
import re
sp.init_printing() 

def latex_to_sympy_matrix(latex_str):
    pattern = r'\\begin\{pmatrix\}(.*?)\\end\{pmatrix\}'
    match = re.search(pattern, latex_str, re.DOTALL)
    if not match:
        raise ValueError("Не удалось найти матрицу в формате pmatrix.")
    
    matrix_content = match.group(1)
    
    rows = re.split(r'\\\\', matrix_content)
    
    matrix = []
    for row in rows:
        row = row.strip().replace('\n', '')
        if not row:
            continue  
        elements = row.split('&')
        numeric_elements = [sp.sympify(elem.strip()) for elem in elements]
        matrix.append(numeric_elements)
    
    return sp.Matrix(matrix)

Задаем значения из своего варианта

In [2]:
latex_A = r"""
    
    
    \begin{pmatrix}
    -3 & 1 & -1\\-1 & 0 & 0\\2 & -2 & 4\\
    \end{pmatrix}
  
  
"""

latex_B = r"""
    
    
    \begin{pmatrix}
    -1 & -1\\-2 & 3\\4 & 0\\
    \end{pmatrix}
  
  
"""

latex_lambd =r"""
    
    
    \begin{pmatrix}
    -1\\-4\\-3\\
    \end{pmatrix}
  
  
"""

latex_r = r"""
    
   
    \begin{pmatrix}
    3\\2\\
    \end{pmatrix}
  
  
"""

A = latex_to_sympy_matrix(latex_A)
B = latex_to_sympy_matrix(latex_B)
lambd = latex_to_sympy_matrix(latex_lambd)
r = latex_to_sympy_matrix(latex_r)


In [3]:
S = B.row_join(A @ B).row_join(A @ A @ B)
print("Матрица управляемости:")
ans = latex(S)
ans = ans.replace("matrix","pmatrix")
ans = ans.replace('\\' + "right]", "")
ans = ans.replace('\\' + "left[", "")
print(ans)
S

Матрица управляемости:
\begin{pmatrix}-1 & -1 & -3 & 6 & -8 & -9\\-2 & 3 & 1 & 1 & 3 & -6\\4 & 0 & 18 & -8 & 64 & -22\end{pmatrix}


⎡-1  -1  -3  6   -8  -9 ⎤
⎢                       ⎥
⎢-2  3   1   1   3   -6 ⎥
⎢                       ⎥
⎣4   0   18  -8  64  -22⎦

Тут, вообще говоря, не совсем понятно, что имеется ввиду под определителем неквадратной матрицы... Судя по методичке и автотестам, определитель на первых трех столбцах...

In [4]:
detS = S[:, :3].det()
print("Определитель матрицы управляемости det(S) = ", detS)
print("Система управляема") if detS != 0 else print("Система не управляема")

Определитель матрицы управляемости det(S) =  -58
Система управляема


In [5]:
lam = sp.symbols('lambda')

F = sp.Matrix([
    [lam - 2, -1, -4],
    [0, lam - 2, 3],
    [0, 0, lam - 5]
])

def compute_D2(matrix):
    minors_2x2 = []
    n = matrix.shape[0]
    
    # Проходим по всем возможным подматрицам 2x2
    for i in range(n):
        for j in range(n):
            submatrix = matrix.minor_submatrix(i, j)  # Удаляем i-ю строку и j-й столбец
            if submatrix.shape == (2, 2):
                minors_2x2.append(submatrix.det())  # Считаем определитель миноров 2-го порядка

    # Находим НОД всех найденных определителей
    D_2 = sp.gcd(minors_2x2)
    return D_2

D_3 = F.det()
D_2 = compute_D2(F)
D_1 = sp.gcd(list(F))
D_0 = 1

E_3 = D_3 / D_2
E_2 = D_2 / D_1
E_1 = D_1 / D_0

def is_non_trivial(polynomial):
    return polynomial != 1

def minimal_control_dimension(E_3, E_2, E_1):
    non_trivial_polynomials = [E_3, E_2, E_1]
    s = sum(1 for poly in non_trivial_polynomials if is_non_trivial(poly))
    return s

s = minimal_control_dimension(E_3, E_2, E_1)

print(f"s = {s}")
if s == 1:
    print("Допускает скалярное управление")
elif s == 0:
    print("Система не управляема")
else:
    print("Не допускает скалярное управление")

s = 1
Допускает скалярное управление


In [6]:
c = A.charpoly().all_coeffs()
c = sp.Matrix(np.flip(c[1:]).reshape((-1, 1)))
print("Коэффициенты характеристического полинома разомкнутой системы:")
ans = latex(c)
ans = ans.replace("matrix","pmatrix")
ans = ans.replace('\\' + "right]", "")
ans = ans.replace('\\' + "left[", "")
print(ans)
c

Коэффициенты характеристического полинома разомкнутой системы:
\begin{pmatrix}-2\\-9\\-1\end{pmatrix}


⎡-2⎤
⎢  ⎥
⎢-9⎥
⎢  ⎥
⎣-1⎦

In [7]:
sigma_1 = np.sum(lambd)
sigma_2 = lambd[0] * lambd[1] + lambd[0] * lambd[2] + lambd[1] * lambd[2]
sigma_3 = np.prod(lambd)

cx = np.zeros(3, dtype=int)
cx[0] = sigma_3 * (-1) ** (3 - 0)
cx[1] = sigma_2 * (-1) ** (3 - 1)
cx[2] = sigma_1 * (-1) ** (3 - 2)

cx = sp.Matrix(cx.reshape((-1, 1)))

print("Коэффициенты характеристического полинома замкнутой системы:")
ans = latex(cx)
ans = ans.replace("matrix","pmatrix")
ans = ans.replace('\\' + "right]", "")
ans = ans.replace('\\' + "left[", "")
print(ans)
cx

Коэффициенты характеристического полинома замкнутой системы:
\begin{pmatrix}12\\19\\8\end{pmatrix}


⎡12⎤
⎢  ⎥
⎢19⎥
⎢  ⎥
⎣8 ⎦

In [8]:
b = B @ r

d3 = b
d2 = A @ d3 + c[2] * b
d1 = A @ d2 + c[1] * b
D = np.hstack((d1, d2, d3))
D = sp.Matrix(D)
D_1 = D.inv()
print("Матрица D^-1:")
ans = latex(D_1)
ans = ans.replace("matrix","pmatrix")
ans = ans.replace('\\' + "right]", "")
ans = ans.replace('\\' + "left[", "")
print(ans)
D_1

Матрица D^-1:
\begin{pmatrix}\frac{30}{929} & - \frac{113}{929} & \frac{25}{1858}\\\frac{48}{929} & \frac{5}{929} & \frac{20}{929}\\- \frac{109}{929} & \frac{8}{929} & \frac{32}{929}\end{pmatrix}


⎡  30   -113    25 ⎤
⎢ ───   ─────  ────⎥
⎢ 929    929   1858⎥
⎢                  ⎥
⎢  48           20 ⎥
⎢ ───   5/929  ─── ⎥
⎢ 929          929 ⎥
⎢                  ⎥
⎢-109           32 ⎥
⎢─────  8/929  ─── ⎥
⎣ 929          929 ⎦

In [9]:
v = (c - cx).T @ D_1 
print("Скалярное управление:")
ans = latex(v)
ans = ans.replace("matrix","pmatrix")
ans = ans.replace('\\' + "right]", "")
ans = ans.replace('\\' + "left[", "")
print(ans)
v

Скалярное управление:
\begin{pmatrix}- \frac{783}{929} & \frac{1370}{929} & - \frac{1023}{929}\end{pmatrix}


⎡-783   1370  -1023 ⎤
⎢─────  ────  ──────⎥
⎣ 929   929    929  ⎦

In [10]:
u = r @ v
print("Векторное управление:")
ans = latex(u)
ans = ans.replace("matrix","pmatrix")
ans = ans.replace('\\' + "right]", "")
ans = ans.replace('\\' + "left[", "")
print(ans)
u

Векторное управление:
\begin{pmatrix}- \frac{2349}{929} & \frac{4110}{929} & - \frac{3069}{929}\\- \frac{1566}{929} & \frac{2740}{929} & - \frac{2046}{929}\end{pmatrix}


⎡-2349   4110  -3069 ⎤
⎢──────  ────  ──────⎥
⎢ 929    929    929  ⎥
⎢                    ⎥
⎢-1566   2740  -2046 ⎥
⎢──────  ────  ──────⎥
⎣ 929    929    929  ⎦